In [1]:
import json
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict

In [ ]:
num_robots = 4
sampling_rate = 5
dataset = 'bathymetry'
budget = True

if budget:
    budget = '_B'
else:
    budget = ''

remove_online = False

filename = f'results/{dataset}_{num_robots}R_{sampling_rate}S{budget}.json'
save_filename = filename[:-5] + '-{}.pdf'
results = json.load(open(filename, 'r'))

rmse = defaultdict(list)
rmse_std = defaultdict(list)

param_time = defaultdict(list)
param_time_std = defaultdict(list)

ipp_time = defaultdict(list)
ipp_time_std = defaultdict(list)

xrange = np.array(list(results.keys())).astype(int)

for num_sensors in results.keys():
    for method in  results[num_sensors].keys():
        for metric in results[num_sensors][method].keys():
            if metric=='RMSE':
                rmse[method].append(np.mean(results[num_sensors][method][metric]))
                rmse_std[method].append(np.std(results[num_sensors][method][metric]))
            elif metric=='ParamTime':
                data = np.array(results[num_sensors][method][metric])
                if 'Adaptive' in method:
                    data /= int(num_sensors)
                param_time[method].append(np.mean(data))
                param_time_std[method].append(np.std(data))
            elif metric=='IPPTime':
                data = np.array(results[num_sensors][method][metric])
                if 'Adaptive' in method:
                    data /= int(num_sensors)
                ipp_time[method].append(np.mean(data))
                ipp_time_std[method].append(np.std(data))

colors = ['C2', 'C3', 'C0', 'C1', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9']
methods = ['Adaptive-SGP', 
           'Adaptive-CMA-ES', 
           'Online-SGP', 
           'Online-CMA-ES']

if sampling_rate == 2 and num_robots == 1 and len(budget) == 0:
    methods.append('Online-BO')
    methods.append('Online-Greedy-MI')
    methods.append('Online-Greedy-SGP')
    methods.append('Online-Discrete-SGP')

'''
# Discrete Online Methods
colors = ['C2', 'C3', 'C1']
methods = ['Online-Discrete-SGP',
           'Online-Greedy-MI', 
           'Online-Greedy-SGP']

# Continuous Online Methods
methods = ['Online-SGP', 
           'Online-CMA-ES', 
           'Online-BO']
'''

plt.figure()
for i, key in enumerate(methods):
    xrange_ = xrange[:len(rmse[key])]
    label = key.strip().replace('Adaptive-SGP', 'Adaptive-SGP (Ours)')
    label = label.replace('CMA-ES', 'CIPP')
    if remove_online:
        label = label.replace('Online-', '')
    plt.plot(xrange_, rmse[key], label=label, c=colors[i])
    plt.fill_between(xrange_,
                     np.array(rmse[key])+rmse_std[key], 
                     np.array(rmse[key])-rmse_std[key], 
                     alpha=0.2,
                     color=colors[i])
plt.legend(loc='upper right')
plt.xlabel("Number of Waypoints")
plt.ylabel("RMSE")
plt.savefig(f'{dataset}_{num_robots}R_{sampling_rate}S{budget}-RMSE.pdf', bbox_inches='tight')
plt.show()

plt.figure()
for i, key in enumerate(methods):
    xrange_ = xrange[:len(ipp_time[key])]
    label = key.strip().replace('Adaptive-SGP', 'Adaptive-SGP (Ours)')
    label = label.replace('CMA-ES', 'CIPP')
    if remove_online:
        label = label.replace('Online-', '')
    plt.plot(xrange_, ipp_time[key], label=label, c=colors[i])
    plt.fill_between(xrange_,
                     np.array(ipp_time[key])+ipp_time_std[key], 
                     np.array(ipp_time[key])-ipp_time_std[key], 
                     alpha=0.2,
                     color=colors[i])
plt.legend(loc='upper left')
plt.xlabel("Number of Waypoints")
plt.ylabel("IPP Runtime (s)")
plt.savefig(f'{dataset}_{num_robots}R_{sampling_rate}S{budget}-IPP.pdf', bbox_inches='tight')
plt.show()